## DB에서 다운도르 받은 민원을 합치고 전처리하여
## 주요 키워드를 확인하는 워크플로우를 통합해놓은 코드 입니다.

순서

1. 여러개의 엑셀파일을 하나의 엑셀로 통합 (저장)

2. 중복제거 및 프리텍스트 제거

3. 토크나이징 (저장)

4. TF값 산출 및 노가다를 통한 클렌징 (저장)

### 엑셀파일 통합

In [2]:
import pandas as pd
import os #파일 경로 획득

In [9]:
file_format = "xlsx" # .csv .xlsx
file_path = "../data/" #본인이 다운로드한 파일이 존재하는 위치로 지정
file_list = [f"{file}" for file in os.listdir(file_path) if file_format in file]

In [10]:
file_list

['PTTN2020.xlsx',
 'SAEOL2021.xlsx',
 'DFPT2022.xlsx',
 'DFPT2021.xlsx',
 'SAEOL2019.xlsx',
 'PTTN2019.xlsx',
 'SAEOL2022.xlsx',
 'PTTN2022.xlsx',
 'DFPT2020.xlsx',
 'SAEOL2020.xlsx',
 'PTTN2021.xlsx',
 'DFPT2019.xlsx']

In [11]:
merged_data = pd.DataFrame()
col_names = pd.read_excel("../data/PTTN2020.xlsx").columns
for file in file_list:
    year = file[-9:-5]
    category = file[:4]
    print(category, year)
    df = pd.read_excel(f"../data/{file}")
    df.columns = col_names
    df['category'] = category
    df['year'] = year
    merged_data =  pd.concat([merged_data, df])
    merged_data = merged_data.reset_index(drop=True)

PTTN 2020
SAEO 2021
DFPT 2022
DFPT 2021
SAEO 2019
PTTN 2019
SAEO 2022
PTTN 2022
DFPT 2020
SAEO 2020
PTTN 2021
DFPT 2019


In [14]:
merged_data.shape

(144814, 30)

save point

In [17]:
# merged_data라는 이름으로 모든 엑셀파일을 통합한 dataframe 생성됨

#merged_data.to_excel("../data/pohang.xlsx") #본인의 도시 명으로 우선 저장
data = pd.read_excel("../data/pohang.xlsx") # 저장한 것 다시 불러와서 작업

In [18]:
data.shape

(144814, 31)

In [21]:
try:
    data = data.drop('Unnamed: 0', axis = 1)
except:
    pass

In [23]:
data.head(2)

,민원신청번호,민원접수번호,제목,본문,처리내용,처리이력,신청일,민원분야,민원발생지,민원신청경로,...,나이,중복민원,관련법령,자민원여부,모민원번호,3회이상종결처리관계,일괄답변모민원번호1,일괄답변모민원번호2,category,year
0,1AA-2001-144354,2AA-2001-165555,[생활불편신고]장애인 전용구역 불법주차,&lt;장애인 전용구역 불법주차&gt; 사진 하단에보시면 구형 사각 주차증입니다 확...,신고주신 차량은 주차가능 표지 미발부자로 2020.1.20 과태료 부과 되어 나갔습...,1. 저희 시 전자민원에 참여해 주셔서 감사합니다.\n\n2. 귀하의 불법주차신고(...,2020-01-08 13:18:31,경찰,경상북도,생활불편신고,...,NaN,N,장애인ㆍ노인ㆍ임산부 등의 편의증진 보장에 관한 법률 제17조(장애인전용주차구역 등...,NaN,NaN,NaN,NaN,NaN,PTTN,2020
1,1AA-2001-371979,2AA-2001-429880,[생활불편신고]장애인 전용구역 불법주차,&lt;장애인 전용구역 불법주차&gt; 너무 상습적이어서 보다보다 신고합니다\n경상...,안녕하세요 노인장애인 복지과 김혜경 입니다.\n\n신고주신 차량은 \n[주차가능] ...,1. 저희 시 전자민원에 참여해 주셔서 감사합니다.\n\n2. 민원업무 답변기한이 ...,2020-01-19 11:46:24,경찰,경상북도,생활불편신고,...,NaN,N,장애인ㆍ노인ㆍ임산부 등의 편의증진 보장에 관한 법률 제17조(장애인전용주차구역 등...,NaN,NaN,NaN,NaN,NaN,PTTN,2020


### 중복제거 및 프리텍스트 제거

In [26]:
data = data.drop_duplicates(['본문']).reset_index(drop = True)
data.shape

(133295, 30)

불법 주정차 관련 민원 제거

In [28]:
import re

pattern = re.compile('불법주차&gt;')

drop_idx = data['본문'].apply(lambda x : pattern.search(str(x)) != None) # 불법주차 패턴 있는 것
normal_idx = data['본문'].apply(lambda x : pattern.search(str(x)) == None) # 불법주차 패턴으로 시작하지 않음

print(f"불법주정차 관련 민원 (제거됨) : {data[drop_idx].shape[0]}")
print(f"일반 민원 : {data[normal_idx].shape[0]}")

불법주정차 관련 민원 (제거됨) : 22515
일반 민원 : 110780


In [32]:
data = data[normal_idx]

프리텍스트 제거

In [33]:
def drop_pretext(x):
    x = x.replace("[민원 유입 경로]: 모바일\n[사고발생지역]", "")
    x = x.replace("G:휴대폰(안전신문고 앱 제외) 또는 PC에 저장된 사진\n* C:안전신문고 앱으로 현장에서 촬영 후 바로 신고한 사진\n* S:안전신문고 앱으로 촬영 및 저장 후 당일 신고한 사진\n * 안전신문고 앱으로 촬영한 사진은 촬영 일시가 자동으로 표기되고, 위·변조 방지기능을 탑재\n\n\n[처리요청기관]:", "")
    x = x.replace("안전신문고 신고파일(사진) 촬영시간 및 경로 안내", "")
    return x

In [34]:
data['본문'] = data['본문'].apply(lambda x :drop_pretext(str(x)))

/tmp/ipykernel_16431/3657969733.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['본문'] = data['본문'].apply(lambda x :drop_pretext(str(x)))


### 토크나이징

In [38]:
from konlpy.tag import Okt

okt = Okt()

In [39]:
def tokenizer(x):
    x = str(x)
    if len(x) < 1000: #너무 긴 민원은 Too Long이라는 키워드로 치환
        return okt.nouns(x)
    else:
        return ["Too Long"]

In [40]:
tokens = data['본문'].apply(lambda x : tokenizer(x))

In [42]:
data['tokens'] = tokens

/tmp/ipykernel_16431/2173657068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tokens'] = tokens


In [44]:
data.head(2)

,민원신청번호,민원접수번호,제목,본문,처리내용,처리이력,신청일,민원분야,민원발생지,민원신청경로,...,중복민원,관련법령,자민원여부,모민원번호,3회이상종결처리관계,일괄답변모민원번호1,일괄답변모민원번호2,category,year,tokens
20,1AA-2004-0268050,2AA-2004-0241554,국세청 감사실 진정서( 국세청에 질의한 것이 왜 울룽군 포항세무서에서답변 하였는지 ...,1. 저가 국세청에 질의 하였고 처리기관은 국세청인데 저의 개인정보를 민원 처리에 ...,1. 안녕하십니까? 항상 국세행정에 대한 관심에 감사드립니다.\n\n2. 귀하께서 ...,1. 안녕하십니까? 항상 국세행정에 대한 관심에 감사드립니다.\n\n2. 귀하께서 ...,2020-04-11 09:19:06,세무,대구광역시,국민신문고 Web,...,N,민원 처리에 관한 법률 제16조(민원문서의 이송),NaN,NaN,NaN,NaN,NaN,PTTN,2020,[Too Long]
22,1AA-2004-0404237,2AA-2004-0353457,불법 외국인 근로자 강력한 단속 조치 요청,사업장 주소 - 포항시 남구 대송면 철강산단로 83(주)삼표피앤씨 협력업체 &quo...,안녕하십니까.\n귀하께서 국민신문고를 통해 제기하신 민원(1AA-2004-04042...,"1. 안녕하십니까. 출입국관리 업무에 관심을 가져주신 것에 대해 감사드리며, 귀하께...",2020-04-16 22:55:49,노동,경상북도,국민신문고 Web,...,N,민원 처리에 관한 법률 제5조(민원인의 권리와 의무),NaN,NaN,NaN,NaN,NaN,PTTN,2020,"[사업, 주소, 포항시, 남구, 대송면, 철강산단로, 주, 표피, 앤, 협력, 업체..."


save point

In [46]:
#data.to_excel("../data/pohang_token.xlsx") # 토크나이징 한 것까지 한번 저장
data = pd.read_excel("../data/pohang_token.xlsx") # 토크나이징 한 것까지 한번 저장

In [48]:
data.shape

(110780, 32)

## 상위 TF 키워드 보며 데이터 정리하기

아래의 셀을 반복적으로 돌려보면서 본인 상황에 맞게 stopwords 추가하기

In [57]:
#분석에서 제거할 키워드를 추가하면 check_freq_tokens()함수에서 데이터 프레임 결과를 보여줄때 제거됨

stopwords = [
    'Too Long', '위반', '차량', '민원', '내용', '관련', '신고', '처리', '해당', '첨부파일', '확인', '문의',
    '다운로드', '요청', '제보', '발생', '경우', '도로', '업무', '주차', '불법', '답변', '사진', '촬영', '안전',
    '신문고', '과태료', '첨부', '저장', '안내', '부과', '갤러리', '불편', '정보', '정차', '동영상', '개인정보',
    '접수'
]

In [52]:
#tf값 계산해주는 함수
from collections import Counter  

def check_freq_tokens(x):
    tf = Counter()
    
    for token in x.tokens:
        tf.update(eval(token))
    tf_df = pd.DataFrame([tf.keys(), tf.values()]).T
    tf_df = tf_df[tf_df[0].apply(lambda x : len(x) > 1)] # 키워드의 길이가 2이상인 것만 표시
    tf_df = tf_df[tf_df[0].apply(lambda x : x not in stopwords)] # 키워드의 길이가 2이상인 것만 표시
    return tf_df.sort_values(1, ascending=False)

In [58]:
check_freq_tokens(data).head(100)

,0,1
3,포항시,82167
217,기관,44847
484,경상북도,41197
505,바로,31151
942,발송,31056
...,...,...
888,구동,8312
1697,직접,8108
964,요망,7898
1484,이송,7308


tf값이 높은 단어들 중 그 단어를 포함한 민원이 어떤 것인지 확인하기 위하여 아래의 셀을 수행 \
예를 들어 tf값 2위인 '기관'은 stopword로 추가할만한 키워드인지 확인하기 위하여 '기관'을 포함한 민원을 확인

In [55]:
def keyword_check(keyword):
    lst_idx = []
    for idx, row in data.iterrows():
        if keyword in row.tokens:
            lst_idx.append(idx)
    return data.iloc[lst_idx, :]

In [59]:
keyword_check('기관')

,Unnamed: 0,민원신청번호,민원접수번호,제목,본문,처리내용,처리이력,신청일,민원분야,민원발생지,...,중복민원,관련법령,자민원여부,모민원번호,3회이상종결처리관계,일괄답변모민원번호1,일괄답변모민원번호2,category,year,tokens
3,25,1AA-2004-0257015,2AA-2004-0263815,학교 구강 검진의 통계는 학교의 행정 사무가 아닌지요?,안녕하세요 \n포항 남구 오천읍에서 작은 치과를 운영하고 있는 채민기입니다. \n저...,1. 안녕하십니까? 귀하께서 국민신문고(2AA-2004-0263815)를 통해 요청...,1. 안녕하십니까? 귀하께서 국민신문고(2AA-2004-0263815)를 통해 요청...,2020-04-10 17:59:46,교육,경상북도,...,N,민원 처리에 관한 법률 제27조(처리결과의 통지),NaN,NaN,NaN,NaN,NaN,PTTN,2020,"['포항', '남구', '오천읍', '치과', '운영', '채민', '기입', '니..."
13,48,1AA-2006-0782074,2AA-2006-0779602,"포항시 북구 삼호로 43 청라빌딩 의 입구 뒤 , 임대광고현수막 앞 인도 , 경...",\n\n\n\n\n\n중앙정부기관 승인 없이\n신청자의 개인정보를\n\n무단 열람하...,"안녕하십니까? \n국민신문고를 통해 포항북부경찰서를 찾아주신 데 감사드리며, \n질...",안녕하십니까?\n 귀하께서 국민신문고를 통해 신청하신 민원에 대한 검토 결과\n다음...,2020-06-26 00:38:57,행정·안전,경상북도,...,N,기타,NaN,NaN,NaN,NaN,NaN,PTTN,2020,"['중앙정부', '기관', '승인', '신청', '자의', '개인정보', '무단',..."
17,56,1AA-2006-0911420,2AA-2007-0026184,(SPP-2006-3003435)6월 2일에 신고했는데 아직 수리가 안되었네요. 철주가,경상북도 포항시 북구 청하면 청계리 29-2\n[기관지정요청]\n6월 2일에 신고했...,해당 통신주에 대해 사업자가 정비완료 하였습니다.,1. 안녕하십니까! 공중케이블 정비 민원센터입니다. \n선생님께서 국민신문고를 통해...,2020-06-30 18:16:02,경찰,경상북도,...,N,기타,NaN,NaN,NaN,NaN,NaN,PTTN,2020,"['경상북도', '포항시', '북구', '청하면', '청계', '리', '기관', ..."
20,62,1AA-2007-0810884,2AA-2007-0813687,화장품책임판매업이 등록되어 있지 않은 불법제품을 사용하여 왁싱샵운영(추가의견),화장품책임판매업이 등록되어 있지 않은 불법제품을 사용하여 왁싱샵운영내용화장품책임판매...,1. 안녕하십니까? 국민신문고를 찾아주셔서 진심으로 감사드립니다.\n\n2. 귀하께...,1. 안녕하십니까? 국민신문고를 찾아주셔서 진심으로 감사드립니다.\n\n2. 고객님...,2020-07-28 04:52:04,산업·통상,강원도,...,N,화장품법,NaN,NaN,NaN,NaN,NaN,PTTN,2020,"['화장품', '책임', '판매업', '등록', '불법', '제품', '사용', '..."
38,83,1AA-2011-0427261,2AA-2011-0447386,(SPP-2011-1503535)어린이 공원에 팻말이 태풍때 떨어졌는데 아직도 나무에,경상북도 포항시 남구 일월동 234-22\n어린이 공원에 팻말이 태풍때 떨어졌는데 ...,1. 귀하께서 국민신문고를 통해 신청하신 민원(신청번호1AA-2011-0427261...,NaN,2020-11-15 14:44:56,행정·안전,경상북도,...,N,기타,NaN,NaN,NaN,NaN,NaN,PTTN,2020,"['경상북도', '포항시', '남구', '월동', '어린이', '공원', '팻말',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110729,133244,1AA-2102-1017205,2AA-2103-0017755,(SPP-2102-2804292)기타 불법주정차 불법 주정차 신고입니다.\n\n* ...,경상북도 포항시 북구 죽도동 59\n기타 불법주정차 불법 주정차 신고입니다.\n\n...,1. 안녕하십니까? 귀하께서 국민신문고를 통해 신청하신 민원(1AA-2102-101...,NaN,2021-02-28 13:18:52,경찰,경상북도,...,N,도로교통법,NaN,NaN,NaN,NaN,NaN,PTTN,2021,"['경상북도', '포항시', '북구', '죽도동', '기타', '불법', '정차',..."
110733,133248,1AA-2102-1003899,2AA-2103-0750473,(SPP-2102-2713118)18:40 동서울발 포항행 우등 고속버스 안이고 좌석번,경상북도 경주시 강동면 다산리 산 53-5\n18:40 동서울발 포항행 우등 고속버...,"1. 안녕하십니까? 평소 교통행정에 관심과 애정을 가져주신 점 깊이 감사드리며, 귀...",NaN,2021-02-27 22:23:03,보건,경상북도,...,N,기타,NaN,NaN,NaN,NaN,NaN,PTTN,2021,"['경상북도', '경주시', '강동면', '다산', '리', '산', '발', '포..."
110734,133249,1AA-2103-0237272,2AA-2103-0266740,(SPP-2103-0702427)(보도블럭 침하 정비요청)\nGS25편의점에서 일품포항물,부산광역시 동래구 안락동 629-28\n(보도블럭 침하 정비요청)\nGS25편의점에...,[주관부서] : 안전도시국 도시안전과 [답변일자] : 2021-03-09 17:23...,NaN,2021-03-07 10:34:05,기타,부산광역시,...,N,NaN,NaN,NaN,NaN,NaN,NaN,PTTN,2021,"['부산광역시', '동래구', '안락동', '보도', '블럭', '침하', '정비'..."
110740,133255,1AA-2103-0538970,2AA-2103-0607023,(SPP-2103-1401598)기타 불법주정차 불법 주정차 신고입니다.\n\n* ...,경상북도 포항시 북구 장성동 1489-1\n기타 불법주정차 불법 주정차 신고입니다....,1. 안녕하십니까? 귀하께서 국민신문고를 통해 신청하신 민원(1AA-2103-053...,NaN,2021-03-14 08:49:54,경찰,경상북도,...,N,도로교통법,NaN,NaN,NaN,NaN,NaN,PTTN,2021,"['경상북도', '포항시', '북구', '장성동', '기타', '불법', '정차',..."


딱히 반복되는 패턴이나 프리텍스트가 보이지 않기 때문에 stopword로 지정하지 않음

### 이러한 방식으로 stopword를 추가한 이후 최종본을 저장

In [60]:
check_freq_tokens(data)

,0,1
3,포항시,82167
217,기관,44847
484,경상북도,41197
505,바로,31151
942,발송,31056
...,...,...
25366,췌장암,1
25368,정환,1
25371,침요,1
25372,로그아웃,1


너무 적게 등장한 단어까지 저장할 필요는 없기 때문에 적당한 선에서 잘라서 저장

In [61]:
tf_df = check_freq_tokens(data)

In [64]:
min_threshold = 10 #열번 이상 등장한 키워드만 남기기

save = tf_df[tf_df[1] >= min_threshold]

In [65]:
save.to_csv("../data/pohang_tf_10.csv")